# STAT4080 Data Programming with Python (online) - Project k nearest neighbours on the TunedIT data set

In [1]:
# Import packages
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import numpy.random as npr

### For the project we will study the method of k nearest neighbours applied to a  music classification data set.These data come from the TunedIT website  http://tunedit.org/challenge/music-retrieval/genres Each row corresponds to a different sample of music from a certain genre. The original challenge was to classify the different genres (the original  prize for this was hard cash!). However we will just focus on a sample of the data (~4000 samples) which is either rock or not. There are 191  characteristics (go back to the website if you want to read about these) The general tasks of this exercise are to: 
- Load the data set - Standardise all the columns 
- Divide the data set up into a training and test set 
- Write a function which runs k nearest neighbours (kNN) on the data set.   (Don't worry you don't need to know anything about kNN) 
- Check which value of k produces the smallest misclassification rate on the    training set 
- Predict on the test set and see how it does

# Q1 
### Load in the data using the pandas read_csv function. The last variable  'RockOrNot' determines whether the music genre for that sample is rock or not

In [2]:
path = '/Users/joshward/Desktop/Fin. Math/Y4S1/Data Programming/'

house_votes = pd.read_csv(path+'house_votes.csv')
tunedit_genres = pd.read_csv(path+'tunedit_genres.csv')

### What percentage of the songs in this data set are rock songs (to 1 d.p.)? 

In [3]:
#since we know it is the last variable as mentioned above we can reference it as [-1]
#values in the column are integers so must be set to integers as they are binary (0,1)

tunedit_genres.iloc[:,-1] = tunedit_genres.iloc[:,-1].astype(int)
percent = round(100*tunedit_genres.iloc[:,-1].sum()/len(tunedit_genres),1)
print("Ans:",percent,"%")

Ans: 48.8 %


# Q2 
### To perform a classification algorithm, you need to define a classification  variable and separate it from the other variables. We will use 'RockOrNot' as  our classification variable. Write a piece of code to separate the data into a  DataFrames X and a Series y, where X contains a standardised version of  everything except for the classification variable ('RockOrNot'), and y contains  only the classification variable. To standardise the variables in X, you need to subtract the mean and divide by the standard deviation

In [4]:
def separate(df,class_column):
    X = df.drop(columns=[class_column])
    y = df.loc[:,class_column]
    X = X.astype(float)
    X = (X - X.mean(axis=0)) / X.std() #standardise variables in X
    return X,y

In [5]:
X,y = separate(tunedit_genres,'RockOrNot')

# Q3 
### Which variable in X has the largest correlation with y?

In [7]:
#use abs because it asks for largest which can be taken as positive or negative

print("Ans:",abs(X.corrwith(y,axis=0)).idxmax())

Ans: PAR_SFM_M


# Q4 
### When performing a classification problem, you fit the model to a portion of  your data, and use the remaining data to determine how good the model fit was. 
### Write a piece of code to divide X and y into training and test sets, use 75% of the data for training and keep 25% for testing. The data should be randomly selected, hence, you cannot simply take the first, say, 3000 rows. If you select  rows 1,4,7,8,13,... of X for your training set, you must also select rows  1,4,7,8,13,... of y for training set. 
### Additionally, the data in the training set cannot appear in the test set, and vice versa, so that when recombined, all data is accounted for. 
### Use the seed 123 when generating random numbers Note: The data may not spilt equally into 75% and 25% portions. In this  situation you should round to the nearest integer.

In [8]:
def divide(seed,X,y,percent_train):
    X_training = X.sample(frac=percent_train,random_state=seed)
    y_training = y.sample(frac=percent_train,random_state=seed)
    
    X_test = X.drop(X_training.index,inplace=False)
    y_test = y.drop(y_training.index,inplace=False)
    
    #We must reindex in order to make the data frames easily accessible
    X_training = X_training.reset_index(drop=True)
    X_test = X_test.reset_index(drop=True)
    y_training = y_training.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)
    
    return X_training,X_test,y_training,y_test

In [9]:
X_training,X_test,y_training,y_test = divide(123,X,y,0.75)

# Q5 
### What is the percentage of rock songs in the training dataset and in the test dataset? Are they the same as the value found in Q1?

In [24]:
#sum the column and divide by the column length to get proportion
train_percent = round(100*(y_training.sum()/len(y_training)),1)
test_percent = round(100*(y_test.sum()/len(y_test)),1)

#I have taken these to be rounded to 1.d.p since we are comparing with Q1 which was asked to be given to 1.d.p

print("The percentage of rock songs in the training dataset is", train_percent,'%')
print("The percentage of rock songs in the test dataset is", test_percent,'%')
print("They are not the same as the value found in Q1 but are both within 2% of the value from Q1 of",percent,"%")

The percentage of rock songs in the training dataset is 49.4 %
The percentage of rock songs in the test dataset is 47.1 %
They are not the same as the value found in Q1 but are both within 2% of the value from Q1 of 48.8 %


# Q6 
### Now we're going to write a function to run kNN on the data sets. kNN works By the following algorithm:
1. Choose a value of k (usually odd)
2. For each observation, find its k closest neighbours
3. Take the majority vote (mean) of these neighbours
4. Classify observation based on majority vote

### We're going to use standard Euclidean distance to find the distance between observations, defined as sqrt( (xi - xj)^T (xi-xj) ) A useful short cut for this is the scipy functions pdist and squareform

### The function inputs are:
1. DataFrame X of explanatory variables 
2. binary Series y of classification values 
3. value of k (you can assume this is always an odd number)

### The function should produce:
1. Series y_star of predicted classification values

In [12]:
#K nearest neighbours algorithm is non-parametric as it identifies k nearest data points to a classification value
#Once it has done this it notes from which explanatory variable

from scipy.spatial.distance import pdist, squareform

def kNN(X,#explanatory variables
        y,#binary series of classification values
        k):#can assume this is always an odd number
    
    n = len(y) #Find the number of obsvervation
    
    y_star = np.zeros(len(y)) #Set up return values
    
    #Calculate the distance matrix for the observations in X
    dist = squareform(pdist(X,'euclidean'),force='tomatrix')
    
    #Make all the diagonals very large so it can't choose itself as a closest neighbour
    dist[np.diag_indices_from(dist)] = 1e10 #not necessary when using argpartition but did it anyway

    y_star = np.zeros(len(X))
    dist = np.argpartition(dist,k,axis=1)
    
    #Loop through each observation to create predictions
    for i in range(len(dist)):
        #Find the y values of the k nearest neighbours. Now allocate to y_star
        y_star[i] = round(y[dist[i,:k]].sum()/k)
        
    y_star = pd.Series(y_star,index=[y.index[i] for i in range(len(y))]) #convert y_star into series
    return y_star

# Q7 
### The misclassification rate is the percentage of times the output of a  classifier doesn't match the classification value. Calculate the  misclassification rate of the kNN classifier for X_train and y_train, with k=3.

In [33]:
def misclass(X,y,k):
    misclassification = 0
    KNN = kNN(X,y,k)
    
    for i in range(len(KNN)):
        if KNN[i] != y[i]:
            misclassification += 1

    return 100*(misclassification/len(KNN))

In [35]:
print("The misclassification rate for k=3 is {}%".format(misclass(X_training,y_training,3)))

The misclassification rate for k=3 is 4.701567189063021%


# Q8 
### The best choice for k depends on the data. Write a function kNN_select that  will run a kNN classification for a range of k values, and compute the  misclassification rate for each.

In [15]:
# The function should produce:
# - a Series mis_class_rates, indexed by k, with the misclassification rates for 
# each k value in k_vals

def kNN_select(X, #- DataFrame X of explanatory variables 
               y, #- DataFrame X of explanatory variables
               k_vals): # - a list of k values k_vals
    mis_class_rates = pd.Series(np.nan,index=k_vals)
    for i in k_vals:
        mis_class_rates[i]=misclass(X,y,i) #this runs a kNN classification within it as instructed
                            
    return mis_class_rates

# Q9
### Run the function kNN_select on the training data for k = [1, 3, 5, 7, 9]  and find the value of k with the best misclassification rate. Use the best  value of k to report the mis-classification rate for the test data. What is  the misclassification percentage with this k on the test set?

In [36]:
k_vals = [1,3,5,7,9] #odd numbers to ensure no tied values

kNN_select(X_training,y_training,k_vals)

1    3.334445
3    4.701567
5    5.201734
7    5.968656
9    6.402134
dtype: float64

In [37]:
print("The best value of k from training data is",kNN_select(X_training,y_training,k_vals).idxmin(), 'which gave a misclassification percent of {}%'.format(min(kNN_select(X_training,y_training,k_vals))),'for the training set')

The best value of k from training data is 1 which gave a misclassification percent of 3.334444814938313% for the training set


In [38]:
print("Ans: {}%".format(min(kNN_select(X_test,y_test,k_vals))),"on the test data which occurs for k=",
      kNN_select(X_test,y_test,k_vals).idxmin())

Ans: 5.0% on the test data which occurs for k= 1


# Q10
### Write a function to generalise the k nearest neighbours classification  algorithm. The function should:
- Separate out the classification variable for the other variables in the dataset,   i.e. create X and y.
- Divide X and y into training and test set, where the number in each is    specified by 'percent_train'.
- Run the k nearest neighbours classification on the training data, for a set    of k values, computing the mis-classification rate for each k
- Find the k that gives the lowest mis-classification rate for the training data,   and hence, the classification with the best fit to the data.
- Use the best k value to run the k nearest neighbours classification on the test   data, and calculate the mis-classification rate
### The function should return the mis-classification rate for a k nearest neighbours classification on the test data, using the best k value for the training data You can call the functions from Q6 and Q8 inside this function, provided they  generalise, i.e. will work for any dataset, not just the TunedIT dataset.

In [19]:
def kNN_classification(df,class_column,seed,percent_train,k_vals):
    # df            - DataFrame to 
    # class_column  - column of df to be used as classification variable, should
    #                 specified as a string  
    # seed          - seed value for creating the training/test sets
    # percent_train - percentage of data to be used as training data
    # k_vals        - set of k values to be tests for best classification
    
    # Separate X and y
    X,y = separate(df,class_column)
    
    # Divide into training and test
    X_training,X_test,y_training,y_test = divide(seed,X,y,percent_train)
    
    # Compute the mis-classification rates for each for the values in k_vals
    missclass = kNN_select(X_training,y_training,k_vals)
        
    # Find the best k value, by finding the minimum entry of mis_class_rates 
    k = missclass.idxmin()
    
    # Run the classification on the test set to see how well the 'best fit'
    kNN(X_test,y_test,k)
    
    # classifier does on new data generated from the same source
    
    
    # Calculate the mis-classification rates for the test data
    mis_class_test = misclass(X_test,y_test,k)
        
    return mis_class_test

### Test your function with the TunedIT data set, with class_column = 'RockOrNot', seed = the value from Q4, percent_train = 0.75, and k_vals = set of k values from Q8, and confirm that it gives the same answer as Q9.

In [39]:
#kNN_classification(df,class_column,seed,percent_train,k_vals)

k_vals = [1,3,5,7,9] #odd numbers to ensure no tied values

kNN_classification(tunedit_genres,'RockOrNot',123,0.75,k_vals)

5.0

### This is the same as our answer from Q9 which was;

In [40]:
print("Ans: {}%".format(min(kNN_select(X_test,y_test,k_vals))),"which occurs for k=",
      kNN_select(X_test,y_test,k_vals).idxmin())

Ans: 5.0% which occurs for k= 1


### Now test your function with another dataset, to ensure that your code  generalises. You can use the house_votes.csv dataset, with 'Party' as the  classifier. Select the other parameters as you wish. This dataset contains the voting records of 435 congressman and women in the  US House of Representatives. The parties are specified as 1 for democrat and 0 for republican, and the votes are labelled as 1 for yes, -1 for no and 0 for abstained. Your kNN classifier should return a mis-classification for the test data (with  the best fit k value) of ~8%.

In [41]:
k_vals = [1,3,5,7,9] #odd numbers to ensure no tied values

kNN_classification(house_votes,'Party',123,0.75,k_vals)

8.256880733944955

In [42]:
print('As we can see this provides a misclassification rate of {}%'.format(kNN_classification(house_votes,'Party',123,0.75,k_vals)),'which is roughly ~8% as predicted')

As we can see this provides a misclassification rate of 8.256880733944955% which is roughly ~8% as predicted
